In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import *
from tkinter import filedialog as fd
from scipy.stats.stats import pearsonr
import os

In [8]:
### Задачи:
## 1) Ввести переменные - загружен ли файл
## 2) Добавить кнопки 
## 3) Мы перебрасываем dataframes из цикла считывания
## 4) Отслеживание через label

In [9]:
COLUMNS = ['YEAR', 'HOUR', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', \
           'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
MONTHS_ARR = COLUMNS[2:]
HOUR_A = np.arange(0,24,1)
MAX_YEAR = 2018
OBS_YEAR = 2019
indices_filename = "ind.txt";
frequences_filename = "freq.txt";

emptyStringChar = 6;

data = pd.DataFrame();
data_tm = pd.DataFrame();

In [10]:
## 
def SaveTable(data, filename):
    file = open(filename, 'w');
    for line in data:
        file.write(str(line));
    file.close()

In [11]:
### Парсинг indices файла
##
def GetIndicesFile():
    fn = fd.askopenfilename()
    if not fn:
        fn = indices_filename;
    indices_filename = fn;
    ParseIndices(fn);
    l1.config(text = "Файл индексов выбран");
    
##
def ParseIndices(file):
    file_content = []
    file_obj = open(file, 'r')
    table = False;
    for line in file_obj:
        if (line.find("=") != -1): 
            if (table):
                table = False;
            else:
                table = True;
        else:
            if table:
                line = line.strip() + "\n";
                file_content.append(line);
    WriteIndices(file_content);
    
##
def WriteIndices(filec):
    for i in range(0,len(filec)):
        filec[i] = filec[i].replace('^', ' ').replace('*', ' ')
    SaveTable(filec, "ind.txt");
    indices_filename = "ind.txt";

In [12]:
##
def FindWords(line):
    mystr = line.replace(' ', '')
    mystr = mystr.replace('.', '')
    mystr = mystr.replace('\n', '')
    t = mystr.isdigit()
    return ((t));

##
def Preproc(filename):
    temp_data_array = [];
    year = 0
    temp_data = open(filename, 'r');
    for line in temp_data:
        if (FindWords(line)):
            if (len(line) < 6):
                year = line.replace('\n', '');
            else:
                line = line.strip() + "\n";
                line = year + " " + line
                temp_data_array.append(line);
    temp_data.close()
    SaveTable(temp_data_array, 'freq.txt');
    frequences_filename = "freq.txt";

##
def GetFrequencesFile():
    fn = fd.askopenfilename()
    if not fn:
        fn = frequences_filename;
    frequences_filename = fn;
    l1.config(text = "Файл частот выбран");
    Preproc(frequences_filename);

In [13]:
##
def addItem():
    word = entry.get();
    stopwords.append(word)
    lbox.insert(END, word)
    entry.delete(0, END)

##
def delList():
    select = list(lbox.curselection())
    select.reverse()
    for i in select:
        lbox.delete(i)
        del(stopwords[i])

In [14]:
## Импорт обработанных данных
def Import():
    data_tm = pd.read_csv(frequences_filename, sep = ' ', \
                     names = COLUMNS, skipinitialspace = True, index_col = 'YEAR')
    fcd = ['YEAR', 'JAN', 'FEB','MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
    data = pd.read_csv(indices_filename, sep = ' ', names = fcd, skipinitialspace = True,  index_col = 'YEAR')
    return data, data_tm;
    
def DropEmpty(data):
    l1.config(text = "Приведение данных к табличному виду");
    for i in data.columns:
        if (i not in COLUMNS):
            data.drop(i, axis = 1, inplace = True)
    return data;

In [15]:
def cube_funct(coefficients, arguments):
    f = (coefficients[0]+coefficients[1]*arguments+\
        coefficients[2]*(arguments**2)+coefficients[3]*(arguments**3))
    return f;

In [16]:
def extractText():
    file_name = fd.asksaveasfilename(filetypes=(("TXT files", "*.txt"),
                                        ("HTML files", "*.html;*.htm"), ("All files", "*.*") ))
    f = open(file_name, 'w')
    s = text.get(1.0, END)
    f.write(s)
    f.close()

In [17]:
def Plotting(t, f2, x, y,  ccs, correlation_coeff, points, MONTHS, HOURS, path, isShow = False, label1 = 'f2(t)', label2 = 'cubic regression'):
    #l1.config(text = "Файл индексов выбран");
    ## Построение фигуры
    ## Размер рисунка
    plt.figure(figsize=(15,10))
    ## Данные
    plt.plot(t, f2,'g^',label=label1, alpha = 0.75)
    plt.plot(x, y, 'r.', label=label2, alpha = 0.75)
    plt.plot(points[0, 0],points[0, 1], 'bD',\
             label = '2019 prediction ({};{})'.format(points[0,0], points[0,1]));
    plt.plot(points[1, 0],points[1, 1], color = 'magenta', marker ='P',\
             label = "2019 factual ({};{})".format(points[1,0], points[1, 1]));
    ## Сетка
    plt.grid(True, color = 'blue',alpha = 0.15)
    ## Тексты:
    plt.annotate("foF2$_m$$_e$$_d$ = ({:.3e})$T^3$ + ({:.3e})$T^2$ + ({:.3e})$T$ + {:.4f}, \n\n\
    Pearson's correlation coefficient: {:.4f}".format(ccs[3],ccs[2],ccs[1],ccs[0], correlation_coeff),\
             xy=(0, 1), xytext=(12, -12), va='top',\
             xycoords='axes fraction', textcoords='offset points', fontsize = 16)
    ## Заголовки
    plt.title("{}, {} o'clock".format(MONTHS, HOURS), fontsize= 20)
    plt.xlabel("Индексы T", fontsize = 16)
    plt.ylabel("Медианные частоты F2, МГц", fontsize = 16)
    ## Размер шрифта легенды
    plt.legend(loc = 'lower right', fontsize = 16)
    ## Команда сохранения и отображения графика
    if not (os.path.exists(path)):
        os.mkdir(path)
    name = path + '/' + MONTHS + '-' + str(HOURS) + '.png'
    plt.savefig(name)
    if isShow:
        plt.show()
    plt.close()

In [18]:
def mainCycle(data, data_tm, path = 'graphs'):
    #print(data.head())
    #print(data_tm.head())
    ## Заготовка таблицы с результатом
    out_table = [];
    out_table.append("Month, Hour, a, b, c, d, Pearson R\n")
    ## Итерируемся по месяцам (столбцам таблицы Tindeces)
    for MON in data.columns:
        # Вторая итерация по часам (То ест мы для каждого месяца делаем итерации по каждому часу)
        for HRS in HOUR_A:
            dF = data_tm.query('HOUR == @HRS & YEAR <= @MAX_YEAR & {} > 0'.format(MON))[MON]
            dT = data.loc[data.index.isin(dF.index)][MON]
            # Извлечение данных из таблицы часть 2
            t = dT.values[:]
            f = dF.values[:]
            ## Програмисстские штуки - перевод в массивы
            t1 = np.array(t);
            f1 = np.array(f);
            ## Запаковка в пары (x, y)
            z = zip(t,f)
            z1 = zip(t1,f1);
            ## Очистка данных от шумов
            f.sort()
            t.sort()
            zs = sorted(z, key = lambda tup: tup[0])
            zs1 = sorted(z1, key = lambda tup: tup[0])
            ## Обратная распаковка очищенных данных
            f = [z[1] for z in zs]
            t = [z[0] for z in zs]
            f1 = [z1[1] for z1 in zs1]
            t1 = [z1[0] for z1 in zs1]
            t1 = np.array(t1)
            t = np.array(t)
            ## определение параметров функции для кубической апроксимации
            #fun = [f[0], f[(len(f)//3)], f[(len(f) * 2//3)], f[len(f)-1]]
            #ft = [t[0], t[(len(t)//3)], t[(len(t) * 2//3)], t[len(t)-1]]
            ## Получение коэффициентов по 4-м точкам (матричный метод кубической аппроксимации)
            #coeff = CalculateCubicCoeff(ft, fun)
            ## Создание сетки области решений
            #lvr = np.arange(t1[0],t1[len(t1)-1], 1)
            ## Построение решения на сетке с полученными коэффициентами
            #cube = cube_funct(coeff,lvr);
            ## Расчет СКО
            #mser_my = mserror(f1, coeff, t1);
            ## Вторая сетка решения на статистически очищенной выборке
            lv = np.arange(-30, t[len(t1) - 1], 1)
            ## Расчет коэффициентов кубического уравнения
            tt = np.polyfit(t, f, 3)
            ## Программистские штуки - разворот вектора (т.к он идет в обратном направлении..)
            tt = list(reversed(tt))
            ## Построение гиперболы
            cube_polyfit = cube_funct(tt,lv)
            corr_coeff = pearsonr(t1, f1)[0]
            accumulated_info = "{},{},{:.5e},{:.5e},{:.5e},{:.5f},{:.5f}\
            \n".format(MON, HRS, tt[3], tt[2], tt[1], tt[0], corr_coeff)
            ## PREDICT
            dFP = data_tm.query('HOUR == @HRS & YEAR == @OBS_YEAR & {} != "NaN"'.format(MON))[MON]
            dTP  = data.loc[data.index.isin(dFP.index)][MON]
            kp = dTP.values[:]
            resp = cube_funct(tt, kp)
            predict = np.array([kp,resp])
            ## для k получили f2(k)
            ## FACT
            dFF = data_tm.query('HOUR == @HRS & YEAR == @OBS_YEAR & {} != "NaN"'.format(MON))[MON]
            dTF  = data.loc[data.index.isin(dFP.index)][MON]
            fact = np.array([dTF.values[:],dFF.values[:]])
            points = np.array([predict,fact])
            # для k получили f2(k)

            out_table.append(accumulated_info)
            Plotting(t1, f1, lv, cube_polyfit, tt, corr_coeff, points, MON, HRS, path, False)
    return out_table;

In [19]:
def SetDependencies():
    data, data_tm = Import();
    data = DropEmpty(data);
    data_tm = DropEmpty(data_tm);
    ot = mainCycle(data, data_tm);
    SaveTable(ot,"result_table.txt");

In [20]:
#ot = mainCycle();
#SaveTable(ot);

In [ ]:
## Параметры окна
root = Tk()
root.title("F2P")
f = Frame()
#lf = LabelFrame(text = 'Слова - исключения {} шт.'.format(len(stopwords)))

## LISTBOX (EXCEPTIONS)
#lbox = Listbox(selectmode=EXTENDED)
#lbox.pack(side=LEFT)
#scroll = Scrollbar(command=lbox.yview)
#scroll.pack(side=LEFT, fill=Y)
#lbox.config(yscrollcommand=scroll.set)

## Поле редактирования исключений
#f.pack(side=LEFT, padx=10)
# entry = Entry(lf)
# badd = Button(lf, text="Добавить", command=addItem)
# bdel = Button(lf, text="Удалить", command=delList)

## Мини-интерфейс
text_info = "Программа запущена"
l1 = Label(bg="lightgreen", width=30, height=5, text = text_info)
bi = Button(text="Загрузить файл индексов", command=GetIndicesFile);
bf = Button(text="Загрузить файл частот", command=GetFrequencesFile);
##bhf = Button(text="Загрузить предобработанный файл частот", command=GetHandledFreqFile);
br = Button(text="Запуск расчета", command = SetDependencies);
#bs = Button(text="Сохранить", command=extractText)

## Упаковка элементов интерфейса
l1.pack(padx = 10, pady = 10);
bi.pack(padx = 10, pady = 10);
bf.pack(padx = 10, pady = 10);
##bhf.pack(padx = 10, pady = 10)
#bs.pack(padx = 10, pady = 10)
br.pack(padx = 10, pady = 10)


root.mainloop()